In [8]:
from transformers import pipeline, DetrConfig, DetrForObjectDetection
from torchvision.transforms import ToTensor
from PIL import Image
import torch

In [2]:
pipeline = pipeline(
    "object-detection", 
    model="facebook/detr-resnet-50",
    dtype=torch.float16,
    device_map=0
)

pipeline("http://images.cocodataset.org/val2017/000000039769.jpg")

/Users/krisnol/.pyenv/versions/detr/lib/python3.11/site-packages/torch/nn/modules/module.py:2446: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/Users/krisnol/.pyenv/versions/detr/lib/python3.11/site-packages/torch/nn/modules/module.py:2446: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/Users/krisnol/.pyenv/versions/detr/lib/python3.11/site-packages/torch/nn/modules/module.py:2446: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a 

[{'score': 0.998046875,
  'label': 'remote',
  'box': {'xmin': 40, 'ymin': 70, 'xmax': 175, 'ymax': 118}},
 {'score': 0.99609375,
  'label': 'remote',
  'box': {'xmin': 333, 'ymin': 72, 'xmax': 368, 'ymax': 187}},
 {'score': 0.99560546875,
  'label': 'couch',
  'box': {'xmin': 0, 'ymin': 1, 'xmax': 639, 'ymax': 473}},
 {'score': 0.9990234375,
  'label': 'cat',
  'box': {'xmin': 13, 'ymin': 52, 'xmax': 314, 'ymax': 471}},
 {'score': 0.99853515625,
  'label': 'cat',
  'box': {'xmin': 345, 'ymin': 23, 'xmax': 640, 'ymax': 368}}]

In [ ]:
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

image_path = "../data/pohang/008820.png"

transform = ToTensor()
pil_image = Image.open(image_path).convert("RGB")
image = transform(pil_image).unsqueeze(0)

with torch.no_grad():
    outputs = model(image)

boxes = outputs.pred_boxes

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-16.4804,   1.1008,  -3.7362,  ..., -15.4978, -12.3444,   9.4784],
         [-15.4036,   1.2846,  -3.6085,  ..., -18.3249, -13.4763,   8.7854],
         [-15.2953,   1.8221,  -3.6223,  ..., -14.6951, -11.2801,   9.0947],
         ...,
         [-15.4489,   2.3088,  -4.4906,  ..., -15.6864,  -9.9195,   8.6375],
         [-16.0143,   0.9093,  -3.2094,  ..., -15.4548, -12.2736,   9.2996],
         [-14.4274,   3.2501,  -3.0114,  ..., -16.1337, -10.3822,   8.1925]]]), pred_boxes=tensor([[[0.2355, 0.5541, 0.0386, 0.0233],
         [0.5911, 0.5711, 0.0329, 0.0336],
         [0.3301, 0.5710, 0.0466, 0.0202],
         [0.5973, 0.5581, 0.0364, 0.0336],
         [0.4688, 0.4677, 0.1012, 0.2568],
         [0.4974, 0.5761, 0.2073, 0.0452],
         [0.5555, 0.5827, 0.0271, 0.0290],
         [0.3574, 0.5615, 0.0383, 0.0369],
         [0.3325, 0.5467, 0.0458, 0.0316],
         [0.3423, 0.5491, 0.0779, 0.0538],
         [0.7373, 0.

In [13]:
outputs.pred_boxes

tensor([[[0.2355, 0.5541, 0.0386, 0.0233],
         [0.5911, 0.5711, 0.0329, 0.0336],
         [0.3301, 0.5710, 0.0466, 0.0202],
         [0.5973, 0.5581, 0.0364, 0.0336],
         [0.4688, 0.4677, 0.1012, 0.2568],
         [0.4974, 0.5761, 0.2073, 0.0452],
         [0.5555, 0.5827, 0.0271, 0.0290],
         [0.3574, 0.5615, 0.0383, 0.0369],
         [0.3325, 0.5467, 0.0458, 0.0316],
         [0.3423, 0.5491, 0.0779, 0.0538],
         [0.7373, 0.6023, 0.0670, 0.0300],
         [0.2427, 0.5712, 0.0581, 0.0425],
         [0.8431, 0.4194, 0.1570, 0.4459],
         [0.8218, 0.5999, 0.0596, 0.0652],
         [0.5617, 0.5914, 0.0291, 0.0178],
         [0.2940, 0.5514, 0.0547, 0.0429],
         [0.2428, 0.5520, 0.0439, 0.0271],
         [0.7237, 0.5377, 0.0878, 0.0459],
         [0.7397, 0.5810, 0.0406, 0.0255],
         [0.7177, 0.5479, 0.0788, 0.0618],
         [0.8361, 0.5994, 0.1541, 0.0887],
         [0.8038, 0.5958, 0.0878, 0.0798],
         [0.9718, 0.5899, 0.0562, 0.0657],
         [0